# 2.字典

## 2.1 字典中的键映射多个值
问题：怎样实现一个键对应多个值的字典 (也叫 multidict )？

解答：将这多个值放到另外的容器中

In [9]:
d = {'a' : [1, 2, 3],
     'b' : [4, 5, 2] 
     } # 使用List保留重复元素

e = {'a' : {1, 2, 3, 2},
     'c' : {7, 6, 9},
     'b' : {4, 5, 2} 
     } # 使用Set去重，并自动排序

使用 collections 模块中的 defaultdict 来构造这样的字典，字典中的value可以指定为list、set等类型：

In [5]:
from collections import defaultdict
d = defaultdict(list)
d['a'].append(1)
d['a'].append(2)
d['b'].append(4) 

In [ ]:
d = defaultdict(set)
d['a'].add(1)
d['a'].add(2)
d['b'].add(4)

## 2.2 字典排序
问题：你想创建一个字典，并且在迭代或序列化这个字典的时候能够控制元素的顺序。


解答：使用 collections 模块中的 OrderedDict 类。在迭代操作的时候它会保持元素被插入时的顺序
- OrderedDict 内部维护着一个根据键插入顺序排序的双向链表。
- 每次当一个新的元素插入进来的时候，它会被放到链表的尾部。
- 对于一个已经存在的键的重复赋值不会改变键的顺序。

In [1]:
from collections import OrderedDict
d = OrderedDict()
d['foo'] = 1
d['bar'] = 2
d['spam'] = 3
d['grok'] = 4 # Outputs "foo 1", "bar 2", "spam 3", "grok 4"
for key in d:
    print(key, d[key])

foo 1
bar 2
spam 3
grok 4


## 2.3 字典的运算

问题：怎样在数据字典中执行一些计算操作 (比如求最小值、最大值、排序等等)？



In [18]:
prices = { 'ACME': 10.75, 'AAPL': 612.78, 'IBM': 205.55, 'HPQ': 37.20, 'FB': 10.75}

如果你在一个字典上执行普通的数学运算，你会发现它们仅仅作用于键，而不是值。比如：

In [3]:
print(min(prices))
print(max(prices))

AAPL
IBM


尝试着使用字典的 values() 方法来解决这个问题，通常这个结果同样也不是你想要的。你可能还想要知道对应的键的信息：


In [5]:
print(min(prices.values()))
print(max(prices.values()))

10.75
612.78


你可以在 min() 和 max() 函数中提供 key 函数参数来获取最小值或最大值对应的键的信息：

In [4]:
print(min(prices, key=lambda k: prices[k]))
print(max(prices, key=lambda k: prices[k]))

FB
AAPL


如果需要同时获取最小值或最大值对应的键和值的信息，可以先使用zip将字典反转成（值，键），再执行min() 和 max() 操作：

In [15]:
reverse = dict(zip(prices.values(), prices.keys()))
print(reverse)
print(prices)

print(min(zip(prices.values(), prices.keys())))
print(max(zip(prices.values(), prices.keys())))

(10.75, 'ACME')
(612.78, 'AAPL')


## 2.4 两字典比较
问题：怎样在两个字典中寻寻找相同点 (比如相同的键、相同的值等等)？

In [21]:
a = {'x' : 1, 'y' : 2, 'z' : 3 }
b = {'w' : 10, 'x' : 11, 'y' : 2}

# Find keys in common
print(a.keys() & b.keys())
# Find keys in a that are not in b
print(a.keys() - b.keys())
# Find (key,value) pairs in common
print(a.items() & b.items())

{'y', 'x'}
{'z'}
{('y', 2)}


注意：尽管字典的 values() 方法也是类似，但是它并不支持这里介绍的集合操作。
某种程度上是因为值视图不能保证所有的值互不相同，这样会导致某些集合操作会出现问题。
不过，如果你硬要在值上面执行这些集合操作的话，你可以先将值集合转换成set，然后再执行集合运算就行了。

## 2.5 从字典中提取子集（字典过滤）
问题：构造一个字典，它是另外一个字典的子集。
解决：（方式一）使用字典推导

In [1]:
prices = { 'ACME': 45.23, 
           'AAPL': 612.78, 
           'IBM': 205.55, 
           'HPQ': 37.20, 
           'FB': 10.75}

# Make a dictionary of all prices over 200
p1 = {key: value for key, value in prices.items() if value > 200}
# Make a dictionary of tech stocks
tech_names = {'AAPL', 'IBM', 'HPQ', 'MSFT'}
p2 = {key: value for key, value in prices.items() if key in tech_names}

print(p1)
print(p2)

{'AAPL': 612.78, 'IBM': 205.55}
{'AAPL': 612.78, 'IBM': 205.55, 'HPQ': 37.2}
